In [174]:
%matplotlib inline
import numpy as np
import numpy.linalg as npln
import sympy as sp
import matplotlib.pyplot as pt

In [340]:
class dist :
    def __init__(self, mean, variance) :
        self.mean = mean
        self.variance = variance
    
    def __repr__(self) : 
        return str("<mean : " + str(self.mean) + ", var : " + str(self.variance) + ">")
        
    # http://stats.stackexchange.com/questions/7440/kl-divergence-between-two-univariate-gaussians
    @staticmethod
    def kl(p, q, sym_p = None, sym_q = None) :
        if (sym_p == None) : 
            (m1, v1), (m2, v2) = (p.mean, p.variance), (q.mean, q.variance)
            m1, v1, m2, v2 = float(m1), float(v1), float(m2), float(v2)
            s1, s2 = np.sqrt(v1), np.sqrt(v2)
            return np.log(s2/s1) + (v1 + (m1 - m2)**2)/(2*v2) - 0.5
        else :
            m1, s1, m2, s2 = sym_p[0], sym_p[1], sym_q[0], sym_q[1]
            return (sp.ln(s2/s1) + (s1**2 + (m1 - m2)**2)/(2*s2**2) - 0.5)
    
    # https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables
    @staticmethod
    def sum(p, q) : 
        return dist(p.mean + q.mean, p.variance + q.variance)
    
    def flip_mean(self) : 
        return dist(-self.mean, self.variance)

In [334]:
class edge :
    def __init__(self, node_a, node_b, dist = dist(1,1), samples = []) :
        self.dist = dist
        self.samples = []
        self.node_a = node_a
        self.node_b = node_b
        # node_b is the greater node
        # dist should be the greater index node - lower index node
        
    def __repr__(self) : 
        return str(self.node_a) + "-" + str(self.node_b)
    
    def has_samples(self) :
        return len(self.samples) != 0
    
    def add_sample(self, sample) :
        self.samples.append(sample)
            
    def weighted_sum_of_squared_errors(self, sym_mean = None, sym_var = None) :
        if len(self.samples) == 0 :
            raise Exception("no samples")
        
        sum_of_squared_errors = 0
        if sym_mean == None :
            mean = self.dist.mean
        else :
            mean = sym_mean
        for s in self.samples :
            sum_of_squared_errors += (s - mean)**2
        
        if sym_var == None : 
            return sum_of_squared_errors / self.dist.variance
        else :
            return sum_of_squared_errors / sym_var
        
    def confidence(self) : 
        if len(self.samples) == 0 : 
            raise Exception("no samples")
        
        err = self.weighted_sum_of_squared_errors()
        n = len(self.samples)
        return n / (np.sqrt(err)+1)
    
    def maximum_likelihood_estimator(self) :
        samples = self.samples
        n = len(samples)
        total = 0.0
        for sample in samples :
            total += sample
        
        mean = total / n
        
        errtotal = 0.0
        for sample in samples : 
            errtotal += (sample - mean)**2
            
        variance = errtotal / n
        return dist(mean, variance)
        

In [347]:
class graph :
    def __init__(self, V) :
        self.__matrix = []
        self.V = V
        # we will use edges only in the upper right triangle
        for i in range(V) :
            self.__matrix.append([])
            for j in range(V) :
                self.__matrix[i].append(edge(j, i))
    
    def __index(self, x, y) :
        if (x == y) :
            raise Exception("no self edges")
        
        if (x > y) : 
            return (y, x)
        
        return (x, y)
    
    def add_sample(self, x, y, sample) :
        x, y = self.__index(x, y)
        self.__matrix[y][x].add_sample(sample)
    
    def set_dist(self, x, y, dist) :
        self.get_edge(x, y).dist = dist
    
    def get_edge(self, x, y) :
        x, y = self.__index(x, y)
        return self.__matrix[y][x]

    def get_sampled_paths(self, x, y) :
        return self.__get_sampled_paths_helper(x, y, {x})
    
    def __get_sampled_paths_helper(self, curr, dest, visited_vertices) :
        if curr == dest :
            return [[]]
        
        total_list_of_paths = []
        for i in range(self.V) :
            if i not in visited_vertices :
                cur_edge = self.get_edge(curr, i)
                if (cur_edge.has_samples()) :
                    new_visited_vertices = visited_vertices.copy()
                    new_visited_vertices.add(i)
                    list_of_paths = self.__get_sampled_paths_helper(i, dest, new_visited_vertices)
                    for path in list_of_paths :
                        path.append(cur_edge)
                    total_list_of_paths += list_of_paths
        
        return total_list_of_paths
        
        

In [360]:
class model :
    @staticmethod
    def path_dist_estimation(path) :
        # path length is always greater than 1.
        estimation = path[0].dist
        
        if path[0].node_a == path[1].node_a or path[0].node_a == path[1].node_b : 
            # node_b - node_a - ... 
            estimation = path[0].dist.flip_mean()
        for i in range(1, len(path)) : 
            curr = path[i]
            prev = path[i-1]
            if curr.node_a == prev.node_a or curr.node_a == prev.node_b :
                estimation = dist.sum(estimation, curr.dist)# head - ... - node_a - node_b - ... - tail
            else :
                estimation = dist.sum(estimation, curr.dist.flip_mean())
            
           
        head = path[0].node_a
        tail = path[-1].node_a
        if path[0].node_a == path[1].node_a or path[0].node_a == path[1].node_b :
            head = path[0].node_b # node_b is the end
            
        if path[-1].node_a == path[-2].node_a or path[-1].node_a == path[-2].node_b :
            tail = path[-1].node_b # node_b is the end
            
        if head > tail :
            return estimation.flip_mean() # 0th is greater than last. SWAP
            
        else : 
            return estimation # last is greater than 0th
    
    @staticmethod
    def path_confidence_estimation(path) : 
        sum_of_reciprocals = 0
        for edge in path :
            sum_of_reciprocals += 1 / edge.confidence()
        
        return np.sqrt(1 / sum_of_reciprocals)
    
    @staticmethod
    def param_cost_formula(graph, x, y, paths = None) :
        edge = graph.get_edge(x, y)
        if (paths == None) :
            paths = graph.get_sampled_paths(x, y)
        if len(paths) == 0 :
            print("no available path between x and y")
            return -1
        
        m1, s1 = sp.symbols('m1 s1')
        cost_sum = 0
        for path in paths :
            if len(path) == 1 : 
                continue
            path_dist = model.path_dist_estimation(path)
            path_confidence = model.path_confidence_estimation(path)
            m2 = path_dist.mean
            s2 = np.sqrt(path_dist.variance)
            cost_sum += dist.kl(0,0,(m1,s1),(m2,s2)) * path_confidence
            #cost_sum += (sp.ln(s2/s1) + (s1**2 + (m1 - m2)**2)/(2*s2**2) - 0.5) * path_confidence
            cost_sum = sp.simplify(cost_sum)
        
        if (edge.has_samples()) :
            mle = edge.maximum_likelihood_estimator()
            n = len(edge.samples)
            confidence = np.sqrt(n / (np.sqrt(n) + 1))
            cost_sum += dist.kl(0,0,(m1,s1), (mle.mean, np.sqrt(mle.variance))) * confidence
        
        return (cost_sum, m1, s1)
    
    @staticmethod
    def param_cost_optimize(formula, m, s, mi, si) :
        formula = sp.simplify(formula)
        # http://homes.soic.indiana.edu/classes/spring2012/csci/b553-hauserk/newtons_method.pdf
        fdm = sp.diff(formula, m)
        fds = sp.diff(formula, s)
        fdmm = sp.diff(fdm, m)
        fdms = sp.diff(fdm, s)
        fdsm = sp.diff(fds, m)
        fdss = sp.diff(fds, s)
        
        gradient_formula = [fdm, fds]
        hessian_formula = [[fdmm, fdms], [fdsm, fdss]]
        
        x = np.array([mi, si])
        counter = 0
        while True :
            gradient, hessian = model.apply_values(gradient_formula, hessian_formula, {m1: x[0], s1: x[1]})
            newx = x - npln.inv(hessian).dot(gradient)
            if np.abs(newx[0] - x[0]) < 0.00001 and np.abs(newx[1] - x[1]) < 0.00001:
                return x[0], x[1]
            
            if counter >= 500 :
                raise Exception('Infinite loop detected')
            x = newx
            counter += 1
        
    
    @staticmethod
    def apply_values(gradient_formula, hessian_formula, sub) : 
        fdm = float(gradient_formula[0].evalf(subs=sub))
        fds = float(gradient_formula[1].evalf(subs=sub))
        fdmm = float(hessian_formula[0][0].evalf(subs=sub))
        fdms = float(hessian_formula[0][1].evalf(subs=sub))
        fdsm = float(hessian_formula[1][0].evalf(subs=sub))
        fdss = float(hessian_formula[1][1].evalf(subs=sub))
        
        return np.array([fdm, fds]), np.array([[fdmm, fdms],[fdsm, fdss]])
    
    @staticmethod
    def optimize_edge(graph, x, y) :
        formula, m1, s1 = model.param_cost_formula(graph, x, y)
        edge = graph.get_edge(x, y)
        mi, si = edge.dist.mean, np.sqrt(edge.dist.variance)
        mean, stddev = model.param_cost_optimize(formula, m1, s1, mi, si)
        #print(mean, stddev**2)
        edge.dist = dist(mean, stddev**2)

In [364]:
g =  graph(4)
g.add_sample(0,1,990)
g.add_sample(0,1,1010)
g.set_dist(0,1,dist(1000,70))
g.add_sample(1,2,1990)
g.add_sample(1,2,2010)
g.set_dist(1,2,dist(2000,50))

g.add_sample(0,3,1190)
g.add_sample(0,3,1210)
g.set_dist(0,3,dist(1200,50))
g.add_sample(3,2,-2390)
g.add_sample(3,2,-2410)
g.set_dist(3,2,dist(-2400,50))
#g.add_sample(0,2,3490)
#g.add_sample(0,2,3510)
g.set_dist(0,2,dist(4000,100))
model.optimize_edge(g,0,2) # somehow forgets all other data. 
print(g.get_edge(0,2).dist)

<mean : 3319.61532154, var : 109.346047135>


In [200]:
model.optimize_edge(g,0,2) # somehow forgets all other data. 
print(g.get_edge(0,2).dist)

float64
float64
float64
<mean : 3.12500479646, var : 0.0999946269742>


In [295]:
m1, s1, m2, s2 = sp.symbols('m1 s1 m2 s2')
kl = 0
kl += sp.ln(s2/s1) + (s1**2 + (m1 - m2)**2)/(2*s2**2) - 0.5
 
sp.diff(kl, m1)

(2*m1 - 2*m2)/(2*s2**2)

In [371]:
a = np.array([0,1])
b = np.array([0,1])
(a == b).all()

True

In [270]:
np.random(1)

TypeError: 'module' object is not callable

In [304]:
1/2.0

0.5

In [305]:
0.0 + 1


1.0

In [310]:
a = (1,2)
a[0]
a[1]

2